In [16]:
import numpy as np
import pandas as pd

In [3]:
def UserSim(n, parameter):
    """
    input :
    n - the number to of users to simulate
    parameter - the rate parameter (lambda)
    
    output: 
    A list of exponential random variable simulations - specifically, the prbability density for each
    simulated random variable instance.
    """
    return list(np.random.exponential(1.0 / parameter, n)) #1st parameter is scale = 1/parameter given aka lambda

In [4]:
def HurdleFun(user_quit_times, breakpoints):
    '''
    user_quit_times: list of times at which user quit
    breakpoints: list of breakpoints
    '''
    user_quit_times = np.sort(user_quit_times)
    total_users = user_quit_times.size
    total_quit_prev = 0
    user_quit_bp = list()
    
    for bp in breakpoints:
        # Get the total users who quit so far
        total_quit = user_quit_times[user_quit_times < bp].size
        # Subtract the total users who quit till previous breakpoint to get users who quit at current breakpoint
        user_quit_bp.append(total_quit - total_quit_prev)
        # Keep track of users who quit so far
        total_quit_prev = total_quit
    
    #Lastly add the remaining users who didn
    remaining_users = total_users - total_quit_prev
    user_quit_bp.append(remaining_users)
    return user_quit_bp
        
HurdleFun([.20, .40], [.25, .5])

[1, 1, 0]

In [5]:
x = [.25, .45, .75]
breaks = [.5]

In [6]:
hf = HurdleFun(x, breaks)
hf

[2, 1]

In [7]:
def exp(lam, x):
    return np.log(1 - np.exp(-1*x*lam))

In [8]:
hf, breaks

([2, 1], [0.5])

In [9]:
def cdf(lam, x):
    '''
    Returns exponential distribution's cdf when lambda and x are given
    '''
    return (1 - np.exp(-1*x*lam))

def EstLam2(hurdles, breaks):
    '''
    Currying function to return another function
    Inputs:
        hurdles: output of HurdleFun
        breaks: list of breakpoints
    Returns: function instance for calcluating log_likelihood given the setup(hurdles, breaks)
    
    TODO: Convert into decorator function
    '''
    
    total_users = sum(hurdles)
    # keep track of m0, m1 and m2
    m0 = hurdles[0]
    bp1 = breaks[0]
    m2 = hurdles[-1]
    bp_last = breaks[-1]
    m1 = total_users - m0 - m2
    
    def log_likehood(lam):
        """
        Specialized function to be called as a lambda, which takes the lam list and
        returns the log_likelihood
        
        """
        log_like = (m0 * np.log(cdf(lam, bp1))) + (m2 * -1*lam*bp_last)
        # If there are users in m1, then add relevant sums to log likelihood
        if m1 != 0:
            for i in range(len(breaks) - 1):
                log_like += hurdles[i+1]*np.log(cdf(lam, breaks[i + 1]) - cdf(lam, breaks[i]))
        return log_like
    
    return log_likehood

In [10]:
p = EstLam2(hf, breaks)

In [11]:
p(1)

-2.3655042591343771

### Part 4

In [12]:
def MaxMLE(survival_list, breakpoints, lambda_list):
    """
    Given the list of survival of users in the form of the output of hurdlefun, breakpoints list and 
    the possible values of lambda, outputs the best lambda for which the MLE estimates are lowest
    Does that by using the EstLam2 function to get the MLE estimate
    
    Input: Survival list of users [], breakpoints [], possible lambda values []
    Output: best lambda float
    """
    PRT = EstLam2(survival_list, breakpoints)
    mle_list = [PRT(x) for x in lambda_list]
    index = np.argmax(mle_list)
    
    return lambda_list[index]

In [13]:
print(MaxMLE( HurdleFun(x, breaks), breaks, list(np.arange(.1, 3, .05))))

2.2


In [26]:
for i, breaks in enumerate([[0.25,0.3],[0.25,0.5],[.25,.75],[.25,3],[.25,50]],1):
    print(i,breaks)

1 [0.25, 0.3]
2 [0.25, 0.5]
3 [0.25, 0.75]
4 [0.25, 3]
5 [0.25, 50]


##### Part 4a.

In [47]:

# Function to estimate lambda using MLE appraoch
def EstLam1(quitting_time): return 1.0/np.mean(np.array(quitting_time))
# Calcualate the difference in the estimates of lambda by EstLam1 and EstLam2
final_df = pd.DataFrame(columns=['breaks', 'lambda1', 'lambda2', 'diff'])
for b, breaks in enumerate([[0.0001,0.3],[0.25,0.3],[.25,1],[.25,5],[.25,10],[2,10],[4,10],[5,10],[.25,5,50],[4,10,50]],1):
    
    lambda_diff = []
    lambda1=[]
    lambda2=[]
    
    for i in range(0,1000):
        
        samples = UserSim(100, 1)
        lmbda1 = EstLam1(samples)
        lmdba2 = MaxMLE(HurdleFun(samples, breaks), breaks, list(np.arange(.1, 3, .05)))
        lambda1.append(lmbda1)
        lambda2.append(lmdba2)
        diff=lmbda1-lmdba2
        lambda_diff.append(diff)
    final_df.loc[b]=[breaks, np.round(np.mean(lambda1),4), np.round(np.mean(lambda2),4), np.round(np.mean(lambda_diff),4)]
print(final_df)

           breaks  lambda1  lambda2    diff
1   [0.0001, 0.3]   1.0101   1.0152 -0.0050
2     [0.25, 0.3]   1.0090   1.0008  0.0083
3       [0.25, 1]   1.0121   1.0093  0.0028
4       [0.25, 5]   1.0080   1.0236 -0.0155
5      [0.25, 10]   1.0111   1.0146 -0.0035
6         [2, 10]   1.0129   1.0159 -0.0029
7         [4, 10]   1.0102   1.3791 -0.3689
8         [5, 10]   1.0111   1.9444 -0.9333
9   [0.25, 5, 50]   1.0157   1.0318 -0.0160
10    [4, 10, 50]   1.0125   1.3598 -0.3473


Moving the breakpoints doesn't effect the estimate of lambda. 